In [1]:
"""
1. Search for an open opportunity at your possible potential clients
2. Search for relevant information from the opening that might act as a good starting point for your business with this company
3. Go through all of your company's projects and find relevant projects
4. Get important to share information about these projects
5. Write a compelling cold email about how your firm can be of great service to your potential client
6. Repeat for ALL potential clients
"""

"\n1. Search for an open opportunity at your possible potential clients\n2. Search for relevant information from the opening that might act as a good starting point for your business with this company\n3. Go through all of your company's projects and find relevant projects\n4. Get important to share information about these projects\n5. Write a compelling cold email about how your firm can be of great service to your potential client\n6. Repeat for ALL potential clients\n"

In [2]:
# Web Scraping

# Relevant information

# Chromadb query

# Email Generation

In [3]:
# Web Scraping
!pipenv install langchain_community

Installing langchain_community...
Installation Succeeded
Installing dependencies from Pipfile.lock (f6673e)...
All dependencies are now up-to-date!
Building requirements...
[    ] Locking packages...
Resolving dependencies...
[    ] Locking packages...
[    ] Locking packages...
[==  ] Locking packages...
[=== ] Locking packages...
[ ===] Locking packages...
[  ==] Locking packages...
[   =] Locking packages...
[    ] Locking packages...
[   =] Locking packages...
[  ==] Locking packages...
[ ===] Locking packages...
[====] Locking packages...
[=== ] Locking packages...
[=   ] Locking packages...
[    ] Locking packages...
[=   ] Locking packages...
[==  ] Locking packages...
[=== ] Locking packages...
[ ===] Locking packages...
[  ==] Locking packages...
[   =] Locking packages...
[    ] Locking packages...
[   =] Locking packages...
[  ==] Locking packages...
[ ===] Locking packages...
[====] Locking packages...
[=== ] Locking packages...
[=   ] Locking packages...
[    ] Locking pac

Loading .env environment variables...
Upgrading langchain_community in  dependencies.


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://meinestadt.jobs.personio.de/job/2259683?language=en&display=de")
page_data = loader.load().pop().page_content
#print(page_data)

In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

#response = llm.invoke("Who is the best footballer of all times?, no preamble")
#print(response.content)

In [14]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    I will give you scraped text from the job posting. 
    Your job is to extract the job details & requirements in a JSON format containing the following keys: 'role', 'experience', 'skills', and 'description'. 
    Only return valid JSON. No preamble, please.
    Here is the scraped text: {page_data}
    """
)

chain_extract = prompt_extract | llm 
response = chain_extract.invoke(input={'page_data':page_data})
print(type(response.content))
print(response.content)

<class 'str'>
```json
{
  "role": "Backend Developer Java (m/f/d) with TypeScript experience",
  "experience": "Experience with Java (Spring), idealerweise also with TypeScript, SQL/noSQL-Datenbanken, and AWS or other Cloud-Plattformen",
  "skills": [
    "Java",
    "TypeScript",
    "SQL/noSQL-Datenbanken",
    "AWS",
    "Cloud-Plattformen",
    "Agile environments",
    "English"
  ],
  "description": "Develop microservices in cloud environments, own your services from development to deployment and operation, take part in architectural decisions and technology selection, and work in agile teams"
}
```


In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_response = json_parser.parse(response.content)
print(type(json_response))

<class 'dict'>


In [18]:
import csv

def read_csv_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        # Skip the header row
        next(csv_reader)
        for row in csv_reader:
            # Separate technical skills (list) and project link (string)
            skills = tuple(row[:-1])  # Exclude the last element (project link)
            project_link = row[-1]  # Get the last element (project link)
            data.append((skills, project_link))  # Create a tuple with skills and link
    return data

# Example usage:
file_path = 'sample_portfolio.csv'  # Replace with your actual file path
data = read_csv_file(file_path)

for skills, project_link in data:
    print(skills, project_link)
    #print(project_link)

('Python', ' SQL', ' Pandas')  https://github.com/user/project1
('SQL', ' Python', ' Airflow')  https://github.com/user/project2
('PySpark', ' Spark SQL', ' Delta Lake')  https://github.com/user/project3
('Machine Learning', ' Deep Learning', ' TensorFlow')  https://github.com/user/project4
('Data Engineering', ' ETL', ' ELT')  https://github.com/user/project5
('Cloud Platforms (AWS', ' GCP', ' Azure)')  https://github.com/user/project6
('Data Warehousing', ' Data Modeling', ' DBT')  https://github.com/user/project7
('Data Visualization', ' Power BI', ' Tableau')  https://github.com/user/project8
('MLOps', ' MLflow', ' Kubeflow')  https://github.com/user/project9
('Natural Language Processing (NLP)', ' NLTK', ' spaCy')  https://github.com/user/project10
('Computer Vision', ' OpenCV', ' TensorFlow')  https://github.com/user/project11
('Time Series Analysis', ' Forecasting', ' Prophet')  https://github.com/user/project12
('Data Cleaning', ' Data Wrangling', ' Pandas')  https://github.com

In [19]:
# Insert data into Vector Database

import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio_links')

if not collection.count():
    for skills, project_url in data:
        collection.add(
            documents=str(skills),
            metadatas={'portfolio_url':project_url},
            ids=[str(uuid.uuid4())]
        )

In [20]:
json_response['skills']

['Java',
 'TypeScript',
 'SQL/noSQL-Datenbanken',
 'AWS',
 'Cloud-Plattformen',
 'Agile environments',
 'English']

In [21]:
portfolio_urls = collection.query(query_texts=json_response['skills'], n_results=2)
portfolio_urls

{'ids': [['2de47c54-ec06-4546-a168-8268967a50c7',
   '4517125d-dd06-4ffe-b99c-b8ba113de71d'],
  ['f4e4f442-23bf-45e9-908a-72361635e916',
   'ec184f79-b4a3-40fa-b361-bc9acd87205e'],
  ['05e3e641-ffaf-4088-9500-a5e11960415d',
   '2de47c54-ec06-4546-a168-8268967a50c7'],
  ['ca2ca369-6141-4116-84ae-db960aed52f3',
   '3fb49e42-a435-4295-98dc-8f3486e1dba3'],
  ['3fb49e42-a435-4295-98dc-8f3486e1dba3',
   'ca2ca369-6141-4116-84ae-db960aed52f3'],
  ['3dc570ba-77f6-43c8-a740-faa34e67c3b6',
   '3fb49e42-a435-4295-98dc-8f3486e1dba3'],
  ['f4e4f442-23bf-45e9-908a-72361635e916',
   '3dc570ba-77f6-43c8-a740-faa34e67c3b6']],
 'embeddings': None,
 'documents': [["('Big Data', ' Hadoop', ' Spark')",
   "('Machine Learning', ' Deep Learning', ' TensorFlow')"],
  ["('Natural Language Processing (NLP)', ' NLTK', ' spaCy')",
   "('Time Series Analysis', ' Forecasting', ' Prophet')"],
  ["('Data Warehousing', ' Data Modeling', ' DBT')",
   "('Big Data', ' Hadoop', ' Spark')"],
  ["('Cloud Platforms (AWS', ' 

In [22]:
job_description = json_response['description']

In [24]:
email_prompt = PromptTemplate.from_template(
    """
    I will give you a role and a task that you have to perform in that specific role.
    Your Role: Your name is Shivaramakrishna, You are an Backend Developer Java (m/f/d) with TypeScript experience
Permanent employee, Full or part time · Germany-wide (mobile)
52,000 - 80,000 € per year
Your mission:
You develop microservices in modern cloud environments (e.g. AWS).

You test and operate your services independently - from CI/CD to monitoring.

You are actively involved in architecture and technology decisions.

You are part of our agile development team: inner team and help design our solutions.

That’s what you bring with you:
You enjoy coding and want to learn.

You have experience with Java (Spring), ideally also with TypeScript.

You are familiar with SQL/noSQL databases.

You have already worked with AWS or other cloud platforms.

You feel comfortable in an agile environment and communicate confidently in English.
. 
    Your Job: Your Job is to write cold emails to clients regarding the Job openings that they have advertised. Try to pitch your clients with an email hook that opens a conversation about a possibility of working with them. Add the most relevant portfolio URLs from
    the following (shared below) to showcase that we have the right expertise to get the job done. 
    I will now provide you with the Job description and the portfolio URLs:
    JOB DESCRIPTION: {job_description}
    ------
    PORTFOLIO URLS: {portfolio_urls}
    """
)

chain_email = email_prompt | llm
response = chain_email.invoke({'job_description':job_description, 'portfolio_urls':portfolio_urls})
print(response.content)

Subject: Expertise in Microservices Development for Cloud Environments

Dear Hiring Manager,

I came across your job posting for a backend developer with expertise in microservices development in cloud environments. As a seasoned Java developer with experience in TypeScript and a passion for working in agile environments, I believe I can bring significant value to your team.

With my background in developing microservices in modern cloud environments, such as AWS, I am confident in my ability to own services from development to deployment and operation. My experience with SQL/noSQL databases and proficiency in Java (Spring) and TypeScript make me a strong fit for this role.

I'd like to highlight a few projects from my portfolio that demonstrate my expertise in microservices development and cloud environments:

* https://github.com/user/project17 (Microservices architecture using Java and Spring)
* https://github.com/user/project20 (Cloud-based data engineering using AWS Glue and Datab